# LeNet Dropout

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from edward.models import Categorical, Normal, Empirical
import edward as ed
import pandas as pd
%matplotlib inline
import csv

In [2]:
mnist = input_data.read_data_sets("MNIST_3_data/",reshape=False, one_hot=True)

Extracting MNIST_3_data/train-images-idx3-ubyte.gz
Extracting MNIST_3_data/train-labels-idx1-ubyte.gz
Extracting MNIST_3_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_3_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Trainable Parameters
learning_rate = 0.001
# num_steps = 2000
batch_size= 64

# Network Parameters
num_classes = 10# MNIST total classes (0-9 digits)
dropout = 0.5 # Dropout, probability to keep units

def lenet(x,n_classes, dropout, reuse, is_training):
    '''
    According to Gal's paper: BAYESIAN CONVOLUTIONAL NEURAL NETWORKS WITH BERNOULLI APPROXIMATE VARIATIONAL INFERENCE
    Author designed bayesian cnn to have dropout after all parameter layers. 
    
    Also, author implemented all all dropout layers have dropout probability of 0.5 
    '''
    # Define a scope for reusing the variables
    with tf.variable_scope('LeNet', reuse=reuse):
        # Convolution layer with 6 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x,32,5,activation=tf.nn.relu)
        #conv1 = tf.layers.dropout(conv1,rate = dropout,training=is_training)

        # max pooling (down-sampling) with strides of 1 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1,1,2)
        
        # Convolution layer with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(conv1,64,5,activation=tf.nn.relu)
        #conv2 = tf.layers.dropout(conv2,rate = dropout,training=is_training)

        # Max Pooling (down-sample) with strides of 1 and kernel-size of 2
        conv2 = tf.layers.max_pooling2d(conv2,1,2)
        
        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)
        
        # Fully connected layer
        fc1 = tf.layers.dense(fc1, 120,activation=tf.nn.relu)
        fc1 = tf.layers.dropout(fc1,rate = dropout,training=is_training)

        # Apply Dropout (if is_training is False, dropout is not applied)
        fc2 = tf.layers.dense(fc1,84,activation=tf.nn.relu)
        fc3 = tf.layers.dropout(fc2,rate = dropout,training=is_training)
        logits = tf.layers.dense(fc3,n_classes)
        return logits
    

In [4]:
x = tf.placeholder(tf.float32,[None,32,32,1])
y_ph = tf.placeholder(tf.int32,[None,10])

logits_train = lenet(x,num_classes,dropout,reuse=False,is_training=True)
logits_test = lenet(x,num_classes,dropout,reuse=True,is_training=False)

# # Predictions
#pred_classes = tf.argmax(logits_test,axis=1)
pred_classes = tf.nn.softmax(logits_test)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_train, 
                                                                        labels=y_ph)
loss_op = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op,
                             global_step=tf.train.get_global_step())
# eval accuracy of model
correct_prediction = tf.equal(tf.argmax(logits_test, 1), tf.argmax(y_ph, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

# Pad images with 0s
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

Image Shape: (28, 28, 1)

Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples
Updated Image Shape: (32, 32, 1)


In [6]:
X_testNew=np.concatenate((X_test,X_validation))
y_testNew=np.concatenate((y_test,y_validation))

In [7]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [8]:
X_test_change = tf.Variable(tf.constant(X_testNew),name="test")
y_test_change = tf.Variable(tf.constant(y_testNew),name="test")

In [9]:
test_number=X_testNew.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_operation = optimizer.minimize(loss_op)
    with open('lenet_partial_dropout_1632.csv', 'a') as f:
        for i in range(20000):
            # batch size is 64
            x_batch, y_batch = next_batch(batch_size,X_train,y_train)

            sess.run(train_operation,feed_dict={x:x_batch,y_ph:y_batch})

            probs=[]
            if i%500==0:
                for samples in range(30):
                    prob =tf.nn.softmax(lenet(X_test_change,num_classes,dropout,reuse=True,is_training=True)).eval()
                    probs+= [prob]
                predict_test=tf.argmax(np.array(probs).sum(axis=0),1)
                accuracy_test=tf.equal(predict_test,tf.argmax(y_testNew,1)).eval().sum()/test_number
                print(accuracy_test)
                writer = csv.writer(f, delimiter=',')
                writer.writerow([accuracy_test])

        
        
        
        
        
        

0.194066666667
0.971866666667
0.978933333333
0.984133333333
0.9846
0.9876
0.9864
0.987466666667
0.9876
0.989333333333
0.989533333333
0.988733333333
0.989333333333
0.9888
0.9896
0.989733333333
0.990666666667
0.991066666667
0.989733333333
0.990866666667
0.990266666667


KeyboardInterrupt: 